In [1]:
import os
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
# Model: ResNet50 without the final classifier
resnet = models.resnet50(pretrained=True)
resnet = torch.nn.Sequential(*(list(resnet.children())[:-1]))  # remove last FC
resnet.to(device)
resnet.eval()

# Image transformation for ResNet
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Imagenet stats
                         std=[0.229, 0.224, 0.225])
])

C:\MyPrograms\Anaconda\envs\tf_gpu\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\MyPrograms\Anaconda\envs\tf_gpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\Safi/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:10<00:00, 10.1MB/s]


In [4]:
def extract_features(dataset_dir, split):
    mapping_path = os.path.join(dataset_dir, split, f"{split}_mapping.csv")
    df = pd.read_csv(mapping_path, header=None, names=["filename", "class"])

    features = []
    labels = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc=f"{split} feature extraction"):
        img_path = os.path.join(dataset_dir, split, row["filename"])
        img = Image.open(img_path).convert("RGB")
        img = transform(img).unsqueeze(0).to(device)

        with torch.no_grad():
            feat = resnet(img).squeeze().cpu().numpy()  # shape: (2048,)
        
        features.append(feat)
        labels.append(row["class"])

    # Save features and labels
    np.save(os.path.join(dataset_dir, split, f"{split}_features.npy"), np.array(features))
    np.save(os.path.join(dataset_dir, split, f"{split}_labels.npy"), np.array(labels))
    print(f"✅ Saved {split} features and labels.")

In [5]:
# Extract for all splits
dataset_path = os.path.join(os.getcwd(), "resnet_dataset")
for split in ["train", "test", "val"]:
    extract_features(dataset_path, split)

train feature extraction: 100%|██████████| 21715/21715 [13:59<00:00, 25.86it/s]


✅ Saved train features and labels.


test feature extraction: 100%|██████████| 6788/6788 [04:30<00:00, 25.13it/s]


✅ Saved test features and labels.


val feature extraction: 100%|██████████| 5417/5417 [03:24<00:00, 26.51it/s]


✅ Saved val features and labels.
